In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import glob
import pandas as pd
import numpy as np
import cudf
import cupy
import gc

from datetime import datetime

In [ ]:
files = glob.glob('/raid/recsys2021_pre_1/*') + glob.glob('/raid/recsys2021_pre_2/*') + glob.glob('/raid/recsys2021_pre_3/*')

In [ ]:
files

In [ ]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"

In [ ]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [ ]:
cluster = LocalCUDACluster(local_directory='/raid/dask6/', device_memory_limit=0.5)
client = Client(cluster)
client

In [ ]:
ddf = dask_cudf.read_parquet(files, columns=['b_user_id'])

In [ ]:
cdf = ddf['b_user_id'].value_counts().reset_index()
cdf.columns = ['b_user_id', 'count']
cdf.to_parquet('/raid/buserid2.parquet')

In [ ]:
ddf = dask_cudf.read_parquet(files, columns=['a_user_id'])

In [ ]:
cdf = ddf['a_user_id'].value_counts().reset_index()
cdf.columns = ['a_user_id', 'count']
cdf.to_parquet('/raid/auserid2.parquet')

In [ ]:
client.close()

In [ ]:
import cudf
import gc

In [ ]:
df1 = cudf.read_parquet('/raid/auserid2.parquet')
df2 = cudf.read_parquet('/raid/buserid2.parquet')

In [ ]:
df = df1.merge(df2, how='outer', left_on='a_user_id', right_on='b_user_id')

In [ ]:
del df1, df2
gc.collect()

In [ ]:
df.columns = ['a_user_id', 'count_a', 'b_user_id', 'count_b']

In [ ]:
df['count_a'] = df['count_a'].fillna(0)
df['count_b'] = df['count_b'].fillna(0)
df['count'] = df['count_a'].fillna(0)+df['count_b'].fillna(0)

In [ ]:
df['user_id'] = df['b_user_id'].copy()
df.loc[df['user_id'].isna(), 'user_id'] = df.loc[df['user_id'].isna(), 'a_user_id'].copy()

In [ ]:
df.to_parquet('abusercount.parquet')

In [ ]:
dfuseremb = df[df['count_b']>7]
dfuseremb = dfuseremb[['user_id', 'count']].reset_index()
dfuseremb.columns = ['drop', 'user_id', 'count']
dfuseremb = dfuseremb.reset_index()
dfuseremb = dfuseremb.drop(['drop'], axis=1)
dfuseremb.columns = ['user_id_', 'user_id', 'count']
dfuseremb['user_id_'] = dfuseremb['user_id_']+1
dfuseremb.to_parquet('./submission/abusercount.parquet')

In [ ]:
dftmp = df[((df['count_a']>8)|(df['count_b']>8))&((df['count_a']>2)&(df['count_b']>2))]

In [ ]:
dftmp = dftmp[['user_id', 'co']].reset_index()
dftmp.columns = ['drop', 'user_id']
dftmp = dftmp.reset_index()

In [ ]:
dftmp = dftmp.drop(['drop'], axis=1)

In [ ]:
dftmp.columns = ['user_id_', 'user_id']

In [ ]:
dftmp['user_id_'] = dftmp['user_id_']+1